### **Install OpenAI**

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_folder='/content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/_eclipse/yellow_eclipse/yellow_eclipse_output'

In [ ]:
from openai import OpenAI
from google.colab import userdata
from IPython.display import Image, display
import requests
from datetime import datetime
import os

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
prompt = "a mythic image of an eclipse, perhaps with an animal of some sort devouring it--please randomly choose a creature that a culture has at one time thought to devour the sun during an eclipse"

response = client.images.generate(
  model="dall-e-3",
  prompt=prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
print(f"Image URL: {image_url}")
display(Image(url=image_url, width=500))

# Fetch the image content
image_content = requests.get(image_url).content

# Generate a timestamp
now = datetime.now()  # current date and time
timestamp = now.strftime("%Y%m%d_%H%M%S")

image_file_path = os.path.join(output_folder, f'eclipse_image{timestamp}.png')

# Create the output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# Save the image
with open(image_file_path, 'wb') as image_file:
    image_file.write(image_content)

print(f"Image saved to {image_file_path}")

In [ ]:
import requests
from google.colab import files

# Assuming this part receives the token successfully as before
from google.colab import userdata
bot_token = userdata.get('SLACK_BOT_TOKEN')

# The channel ID you want to send the message to
channel_id = userdata.get('slack_channel_id')

# File path in Google Colab environment
file_path = image_file_path # Correct path according to your file location

# Prepare the data payload for multipart/form-data
data = {
    'channels': channel_id,  # Channel ID the file is posted to
    'filename': 'eclipse_image{timestamp}.png',  # Filename as it will appear on Slack
    'title': 'Eclipse Image Upload',  # Title of file
}

# Note: 'files' parameter is used to upload the actual file content
files = {
    'file': (file_path, open(file_path, 'rb'), 'png'),
}

# Headers - note that 'Content-Type' is not needed here, as requests will automatically
# use 'multipart/form-data' for the 'files' parameter
headers = {
    'Authorization': f'Bearer {bot_token}',
}

# Post the file to Slack
response = requests.post('https://slack.com/api/files.upload',
                         headers=headers, data=data, files=files)

# Checking and reporting the status of the file upload
if response.status_code != 200:
    raise ValueError(
        f'Request to slack returned an error {response.status_code}, the response is:\n{response.text}'
    )
else:
    print("File uploaded successfully.")

# Make sure to close the file after uploading
files['file'][1].close()